<a href="https://www.kaggle.com/code/tusharaggarwal27/titanic-survivor-predict-with-lightgbm?scriptVersionId=124030852" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="color:white;
           display:fill;
           border-radius:20px;
           background-color:#F5DEB3;
           font-size:170%;
           font-family:Helvetica, Sans-Serif;
           letter-spacing:0.5px">
<p style="padding: 10px;
          text-align: center;
          font-size:150%;
          color:blue;">
          🚢🛳️((Titanic Survivor Predict with LightGBM))🚢🛳️
            
</p>
<style>
        h1{text-align: center;}
</style>  
    
</div>

<p style="font-family:cursive; font-size:18px; color:'darkcyan';">I brewed this notebook from scratch, If this notebook helped, please consider upvoting and cite me if sharing ,Thank you!</p>

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:18px;border-radius:20px">
    <a href="https://www.linkedin.com/in/tusharaggarwalinseec/" target="_blank">Lets connect on LinkedIn!</a>
    
   </p>
<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:18px;border-radius:20px">
<a href="https://github.com/tushar2704" target="_blank">Follow me on Github too!</a> </p>
<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:18px;border-radius:20px">
    <a href="https://medium.com/@tushar_aggarwal" target="_blank">Also checkout my Medium posts!</a>
    
   </p>

<div class="alert alert-info" role="alert" style="font-size:17px;border-radius:20px">
    <b>In this project, I made a model with high accuracy to predict the Titanic survivor with LightGBM,</b>
    <br><br>
    <b>The Challenge:
        <br>
The sinking of the Titanic is one of the most infamous shipwrecks in history.</b>
    
</div>
<div class="alert alert-info" role="alert" style="font-size:15px;border-radius:20px">
    <b>Some things to note:</b>
    <br><br>On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.
    <br><br>
    In this challenge, i have to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).
  
</div>



<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Importing the required Libraries
    
   </p>

In [ ]:
import seaborn as sns
import sys
import csv
import datetime
import operator
import joblib
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from statsmodels.formula.api import ols
from sklearn.metrics import cohen_kappa_score
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy.stats import norm, skew, probplot

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Reading the Dataset
    
   </p>

In [ ]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">EDA for the Data
    
   </p>

In [ ]:
df_train.head(5)

In [ ]:
df_test.head(5)

In [ ]:
gender_submission.head(5)
# When using head(), it can be expected that the result of test data can be predicted through PassengerId.

In [ ]:
# Check if the 'Survived' column is attached properly.
df_test['Survived'] = gender_submission['Survived']
df_test.head(5)

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">EDA with Pairplot (for multiple pairwise bivariate distributions)
    
   </p>


In [ ]:
# Let's analyze how the distribution is statistically through pair plots and several graphs.
sns.pairplot(data = df_train)

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">The first graph now is the result of heatmap analysis of data. The higher the correlation, the darker the color.
    <br>The second graph graphed the null ratio.
    
   </p>

In [ ]:
def analysis(data):
    sns.heatmap(data.corr(), annot=True, cmap='Reds')
    null_percent = 100*(data.isnull().sum()/len(data))
    null_percent = null_percent[null_percent>0].sort_values()
    plt.figure(figsize= (10,4))
    sns.barplot(x=null_percent.index, y= null_percent)
    plt.xticks(rotation=90)

In [ ]:
#For df_train
analysis(df_train)

In [ ]:
#For df_test
analysis(df_test)

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now using OLS Regression
   </p>

In [ ]:
fat=ols(formula='Survived~PassengerId+Pclass+Age+Sex+SibSp+Parch+Fare+Embarked', data=df_train).fit()
print(fat.summary())
# Statistically, it was found that the survival rate was the most influential according to the classification according to gender and Pclass.
# When predicting with OLS regression, if we consider only the above 8 variables, it was expected that the R-squre 0.4/ Adjust R-square 0.39 level could be predicted.

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now numeric dtype Feature Enginearing
   </p>


In [ ]:
num_cols = [col for col in df_train.columns if df_train[col].dtype in ['int64','float64']]

df_train[num_cols].describe()

In [ ]:
#Now, finfind correlation for df_train
df_train.corr(method='pearson')

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, checking for duplicate values in the data
   </p>


In [ ]:
#Checking metadata for df_train
df_train.info()

In [ ]:
#Dropping duplicates
df_train=df_train.drop_duplicates()

In [ ]:
#Checking for difference
df_train.info()

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, checking for skewness and kurtosis
   </p>
​

In [ ]:
# When checking skewness and kurtosis, it is often seen in Regression Modeling, but in this case, we could not simply obtain information other than that the distribution of Y-value (Survived) we wanted to know was 3:2.
print(f'skew: {df_train.Survived.skew()}')
print(f'kert: {df_train.Survived.kurt()}')
sns.distplot(df_train.Survived, fit = norm)
f = plt.figure()
probplot(df_train.Survived, plot = plt)
plt.show()

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, Countplots
   </p>


In [ ]:
def count_plot(d, y, x):
    plt.figure(figsize=(12,6))
    sns.countplot(x = d[y], hue = x, data=d)
    plt.ylabel('Number of people')
    plt.title('Survival count by '+ x)

In [ ]:
numeric_cols = [col for col in df_train if df_train[col].dtype in ['int64','float64']]
numeric_cols.remove('Survived')
y = 'Survived'
over_column_name = list()

for i in numeric_cols:
    if (len(df_train[i].value_counts())<20):
        count_plot(df_train, y, i)
    elif (len(df_train[i].value_counts())>20):
        over_column_name.append(i)

print('Columns with more than 20 variables in the column are as shown below.The distribution of numbers of columns and survivors with less than 20 variables is as shown in the graph.')
print(over_column_name)

In [ ]:
print(num_cols)
num_cols.remove('PassengerId')

fig, ax = plt.subplots(3, 2, figsize=(20, 10))

for variable, subplot in zip(num_cols, ax.flatten()):
    sns.countplot(df_train[variable], ax=subplot)
    for label in subplot.get_xticklabels():
        label.set_rotation(60)
        
plt.tight_layout()

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, For Category columns
   </p>

In [ ]:
cat_cols = [col for col in df_train if df_train[col].dtype not in ['int64','float64']]
print(cat_cols)
cat_cols.remove('Name')

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

for variable, subplot in zip(cat_cols, ax.flatten()):
    sns.countplot(df_train[variable], ax=subplot)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
        
plt.tight_layout()

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, Checking for missing values and dropping them
   </p>

In [ ]:
df_train.isna().sum()

In [ ]:
#Dropping missing values
df_train=df_train.drop(['Name','Ticket'],axis=1)
df_test=df_test.drop(['Name','Ticket'],axis=1)

In [ ]:
# Cabin It's a waste to throw away all the information, so I'm thinking of using the number of characters as a variable.
# Rather than deleting the Cabin column, I will use "len()" to use the number of characters in the variable.
df_train['CabinCode'] = df_train['Cabin'].apply(lambda x : len(str(x)) if x!='nan' else 0)
df_test['CabinCode'] = df_test['Cabin'].apply(lambda x  : len(str(x)) if x!='nan' else 0)

In [ ]:
df_train=df_train.drop(['Cabin'],axis=1)
df_test=df_test.drop(['Cabin'],axis=1)

In [ ]:
# Assuming that the nan value is also a variable, in this analysis, we will proceed without correction for the null value.

df_train=pd.get_dummies(df_train)
df_test=pd.get_dummies(df_test)

In [ ]:
df_train.info()
df_test.info()

In [ ]:
df_train.head(5)

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, Building the Model - LGBMClassifier
   </p>

In [ ]:
from sklearn.model_selection import train_test_split

random_state_val =42 #Choosing this value
test_size_val =0.3
train,validation = train_test_split(df_train, test_size = test_size_val, random_state = random_state_val)

In [ ]:
#Defining variables
drop_col = ['Survived']
y_nm = 'Survived'

df_train_x = train.drop(drop_col, axis = 1)
df_train_y = pd.DataFrame(train[y_nm])

df_val_x = validation.drop(drop_col, axis = 1)
df_val_y = pd.DataFrame(validation[y_nm])

df_test_x = df_test.drop(drop_col, axis = 1)
df_test_y = pd.DataFrame(df_test[y_nm])

In [ ]:
#Now creating LGBMClassifier and adjusting hyperparas
LGBClassifier = lgb.LGBMClassifier(objective='binary',
                                   max_depth = 8,
                                   learning_rate = 0.01,
                                   n_estimators = 9000,
                                   max_bin = 200,
                                   bagging_freq = 4,
                                   bagging_seed = 8,
                                   feature_fraction = 0.2,
                                   feature_fraction_seed = 8,
                                   min_sum_hessian_in_leaf = 11,
                                   verbose = -1,
                                   random_state = 42)

In [ ]:
#Fitting the model on LGBMClassifier
start = datetime.datetime.now()
lgbm = LGBClassifier.fit(df_train_x.values,
                       df_train_y.values.ravel(),
                       eval_set = [(df_train_x.values, df_train_y), (df_val_x.values, df_val_y)],
                       eval_metric ='logloss',
                       early_stopping_rounds = 20,
                       verbose =False)
end = datetime.datetime.now()
end-start

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, Checking the variables that have a major impact, and graphing them.
   </p>

In [ ]:
feature_imp= pd.DataFrame(sorted(zip(lgbm.feature_importances_, df_test_x.columns), reverse = True), columns = ['Value', 'Feature'])
feature_imp.to_excel("feature_imp.xlsx")

plt.figure(figsize=(7,5))
sns.barplot(x='Value', y='Feature', data=feature_imp.sort_values(by='Value', ascending=False))
plt.tight_layout()
plt.show()
plt.savefig('lightGBM_ Importances.png')

In [ ]:
# I put the logic that yields the highest accuracy using a for loop.

result_lst =[]
max_accuracy =0.
opt_threshold =0.
val_y_prob = lgbm.predict_proba(df_val_x.values)[:, 1]

for n in range(0,60):
    threshold = round(((n+1)*0.01),2)
    pred_yn = val_y_prob.copy()
    pred_yn = np.where(pred_yn > threshold, 1., 0.)
    
    result_dict = {}
    precision, recall, f1_score, support = precision_recall_fscore_support(df_val_y.values.ravel(), pred_yn, average='binary')
    accuracy = accuracy_score(df_val_y.values.ravel(), pred_yn)
    kappa = cohen_kappa_score(df_val_y.values.ravel(), pred_yn)
    
    result_dict ={'Threshold': threshold, 'Accuracy': round(accuracy,4), 'Precision': round(precision,4), 'Recall': round(recall,4), 'F1_Score': round(f1_score,4), 'Kappa': round(kappa,4)}
    result_lst.append(result_dict)
    
    if max_accuracy <= accuracy:
        max_accuracy = accuracy
        opt_threshold = threshold
        
    confMat = confusion_matrix(df_val_y.values.ravel(), pred_yn, labels=[1,0])
    
matric_df = pd.DataFrame(result_lst, columns=['Threshold','Accuracy', 'Precision', 'Recall', 'F1_Score', 'Kappa'])
matric_df.to_csv('REC_scores.csv',sep=',', header=True, index=False, encoding='UTF-8')

print('Best Accuracy-SCORE =%f, Threshold=%f'%(max_accuracy, opt_threshold))
print('Threshold Setup complete')

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, Analyzing the model results,
    <br><br>
I tried to implement the results by directly calculating the train, validation, and test results (from precision to F1-Score, AUROC) through the model.
   </p>


In [ ]:
#Predicting on train
predict_lgbm = lgbm.predict_proba(df_train_x.values)[:,1]
pred_train = np.where(predict_lgbm > opt_threshold, 1., 0.)
tp, fn, fp, tn = confusion_matrix(df_train_y.values.ravel(), pred_train, labels=[1,0]).ravel()

In [ ]:
#Confusion Matrix
conf_matrix = pd.DataFrame(
    confusion_matrix(df_train_y.values.ravel(), pred_train),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(df_train_y.values.ravel(), pred_train))

In [ ]:
#Plotting ROC Curve

from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(df_train_y.values.ravel(), predict_lgbm)

import matplotlib.pyplot as plt
roc_auc = auc(fpr, tpr)

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">First, Model Metric Sumamry
   
   </p>

In [ ]:
Accuracy_Rate = (tp + tn) / (tp + tn + fp + fn)
Recall_Rate = tp / (tp + fn)
Precision_Rate = tp / (tp + fp)
Specificity_Rate = tn / (tn + fp)
F1_Score = (Precision_Rate * Recall_Rate) / (Precision_Rate + Recall_Rate) * 2

print(" - Accuracy Rate    : {:2.3f} %".format(Accuracy_Rate*100))
print(" - Recall Rate      : {:2.3f} %".format(Recall_Rate*100))
print(" - Precision Rate   : {:2.3f} %".format(Precision_Rate*100))
print(" - Specificity Rate : {:2.3f} %".format(Specificity_Rate*100))
print(" - F1 Score         : {:2.3f} ".format(F1_Score*100))
print(" - ROC AUC          : {:2.3f} ".format(roc_auc*100))

In [ ]:
#Predicting on Validation data
predict_lgbm = lgbm.predict_proba(df_val_x.values)[:,1]
pred_val = np.where(predict_lgbm > opt_threshold, 1., 0.)

tp, fn, fp, tn = confusion_matrix(df_val_y.values.ravel(), pred_val, labels=[1,0]).ravel()

conf_matrix = pd.DataFrame(
    confusion_matrix(df_val_y.values.ravel(), pred_val),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(df_val_y.values.ravel(), pred_val))

from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(df_val_y.values.ravel(), predict_lgbm)

import matplotlib.pyplot as plt
roc_auc = auc(fpr, tpr)

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

Accuracy_Rate = (tp + tn) / (tp + tn + fp + fn)
Recall_Rate = tp / (tp + fn)
Precision_Rate = tp / (tp + fp)
Specificity_Rate = tn / (tn + fp)
F1_Score = (Precision_Rate * Recall_Rate) / (Precision_Rate + Recall_Rate) * 2

print("Second, Model Metric Sumamry")
print(" - Accuracy Rate    : {:2.3f} %".format(Accuracy_Rate*100))
print(" - Recall Rate      : {:2.3f} %".format(Recall_Rate*100))
print(" - Precision Rate   : {:2.3f} %".format(Precision_Rate*100))
print(" - Specificity Rate : {:2.3f} %".format(Specificity_Rate*100))
print(" - F1 Score         : {:2.3f} ".format(F1_Score*100))
print(" - ROC AUC          : {:2.3f} ".format(roc_auc*100))

In [ ]:
predict_lgbm = lgbm.predict_proba(df_test_x.values)[:,1]
pred_test = np.where(predict_lgbm > opt_threshold, 1., 0.)

tp, fn, fp, tn = confusion_matrix(df_test_y.values.ravel(), pred_test, labels=[1,0]).ravel()

conf_matrix = pd.DataFrame(
    confusion_matrix(df_test_y.values.ravel(), pred_test),
    columns=['Predicted Value 0', 'Predicted Value 1'],
    index=['True Value 0', 'True Value 1']
)

print("1. Counfusion Matrix")
print(conf_matrix.T)
print("")

print("2. Classification Report")
print(classification_report(df_test_y.values.ravel(), pred_test))

from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(df_test_y.values.ravel(), predict_lgbm)

import matplotlib.pyplot as plt
roc_auc = auc(fpr, tpr)

# Plot of a ROC curve for a specific class
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

Accuracy_Rate = (tp + tn) / (tp + tn + fp + fn)
Recall_Rate = tp / (tp + fn)
Precision_Rate = tp / (tp + fp)
Specificity_Rate = tn / (tn + fp)
F1_Score = (Precision_Rate * Recall_Rate) / (Precision_Rate + Recall_Rate) * 2

print("Third, Model Metric Sumamry")
print(" - Accuracy Rate    : {:2.3f} %".format(Accuracy_Rate*100))
print(" - Recall Rate      : {:2.3f} %".format(Recall_Rate*100))
print(" - Precision Rate   : {:2.3f} %".format(Precision_Rate*100))
print(" - Specificity Rate : {:2.3f} %".format(Specificity_Rate*100))
print(" - F1 Score         : {:2.3f} ".format(F1_Score*100))
print(" - ROC AUC          : {:2.3f} ".format(roc_auc*100))

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, Preparing submission materials
   
   </p>

In [ ]:
test_result= pd.DataFrame(pred_test)
test_result.columns = ['Survived']
predict = test_result['Survived']
Id_No = df_test['PassengerId']
submission = pd.DataFrame({'PassengerId': Id_No, "Survived": predict})
submission['Survived'] = submission['Survived'].astype('Int64')
submission.to_csv('submission.csv', index=False)
submission.head()

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:17px;border-radius:20px">Now, Submitted the csv for the competition

<div style="color:black;
           display:fill;
           border-radius:5px;
           background-color:#a5d610;
           font-size:120%;
           font-family:Helvetica, Sans-Serif;
           letter-spacing:0.5px">

<p style="padding: 10px;
          text-align: center;
          color:black;">
We are doing something right here, 😀, as after submission rank I got is under Top 12% 😀!!, follow me for more optimization on this and better results
</p>

<p style="font-family:cursive; font-size:18px; color:'darkcyan';">I brewed this notebook from scratch, If this notebook helped, please consider upvoting and cite me if sharing ,Thank you!</p>

<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:18px;border-radius:20px">
    <a href="https://www.linkedin.com/in/tusharaggarwalinseec/" target="_blank">Lets connect on LinkedIn!</a>
    
   </p>
<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:18px;border-radius:20px">
<a href="https://github.com/tushar2704" target="_blank">Follow me on Github too!</a> </p>
<p style="width: 700px;padding: 20px;background: papayawhip;border-radius:10px;font-size:18px;border-radius:20px">
    <a href="https://medium.com/@tushar_aggarwal" target="_blank">Also checkout my Medium posts!</a>
    
   </p>